In [ ]:
#pip install pandas scikit-learn streamlit openpyxl


Note: you may need to restart the kernel to use updated packages.


# Sviluppo di un Modello di Regressione e Web App per la Predizione dei Prezzi Immobiliari

Sviluppare un modello di regressione per predire il prezzo al metro quadro di immobili nella regione di Sindian, Nuova Taipei, Taiwan, utilizzando il Real Estate Valuation Data Set. Successivamente, creare una web app con Streamlit che permetta agli utenti di ottenere una stima del prezzo inserendo:

• Latitudine e longitudine, oppure

• Età dell’immobile, distanza dalla stazione MRT più vicina e numero di minimarket nelle vicinanze (opzionale).

Requisiti Minimi:

1. Sviluppo del Modello:

• Costruire un modello di regressione per predire il prezzo al metro quadro basato su latitudine e longitudine.

2. Web App con Streamlit:

• Creare un’interfaccia che consenta agli utenti di inserire latitudine e longitudine per ottenere una stima del prezzo.

• Validare che i valori inseriti siano entro i limiti del dataset.



In [ ]:
import pandas as pd 
df = pd.read_excel("./Real estate valuation data set.xlsx") 


The inputs are as follows

- X1=the transaction date (for example, 2013.250=2013 March, 2013.500=2013 June, etc.)
- X2=the house age (unit: year)
- X3=the distance to the nearest MRT station (unit: meter)
- X4=the number of convenience stores in the living circle on foot (integer)
- X5=the geographic coordinate, latitude. (unit: degree)
- X6=the geographic coordinate, longitude. (unit: degree)

The output is as follow
Y= house price of unit area (10000 New Taiwan Dollar/Ping, where Ping is a local unit, 1 Ping = 3.3 meter squared)



In [5]:
df.drop(columns = ['No'], inplace = True)

In [12]:
df.rename(columns={
    'X1 transaction date': 'transaction_date',
    'X2 house age': 'house_age',
    'X3 distance to the nearest MRT station': 'mrt_distance',
    'X4 number of convenience stores': 'convenience_stores',
    'X5 latitude': 'latitude',
    'X6 longitude': 'longitude',
    'Y house price of unit area': 'price_per_unit_area'
}, inplace=True)

# Verifica
print(df.columns)

df.head() #pulito
df.isna().sum() #pulito
df.tail() #pulito 
print(df.info()) 
df.shape #414 obs per 6 variabili esplicative quantitative  e 1 target quantitativo 


Index(['transaction_date', 'house_age', 'mrt_distance', 'convenience_stores',
       'latitude', 'longitude', 'price_per_unit_area'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   transaction_date     414 non-null    float64
 1   house_age            414 non-null    float64
 2   mrt_distance         414 non-null    float64
 3   convenience_stores   414 non-null    int64  
 4   latitude             414 non-null    float64
 5   longitude            414 non-null    float64
 6   price_per_unit_area  414 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 22.8 KB
None


(414, 7)

## IMPLEMENTAZIONE REGRESSIONE

y = "price_per_unit_area"  

X = ['latitude', 'longitude']

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import pickle

In [16]:
# Feature e target
X = df[['latitude', 'longitude']]
y = df['price_per_unit_area']

# Modello
model = LinearRegression()

# Cross-validation a 10 fold (R² score)
scores = cross_val_score(model, X, y, cv=10, scoring='r2')
print(f"R² Cross-Validation (10-fold): {scores.mean():.3f} ± {scores.std():.3f}")


R² Cross-Validation (10-fold): 0.392 ± 0.117


### Giustificazione dell'adozione di un modello più complesso

Il coefficiente di determinazione \( R^2 = 0.39 \) ottenuto con regressione lineare e validazione incrociata a 10 fold indica che il modello spiega solo il **39% della variabilità** nel prezzo al metro quadro in funzione della latitudine e longitudine.

Statisticamente, questo suggerisce una relazione **linearmente debole** tra posizione geografica (lat/lon) e prezzo.  
Commercialmente, ciò significa che **non possiamo fare stime affidabili del prezzo immobiliare considerando solo le coordinate utilizzando un modello di regressione lineare**: altri fattori (età dell'immobile, distanza dai servizi, infrastrutture locali, ecc.) giocano un ruolo fondamentale nella determinazione del valore di mercato. 

Pertanto, si rende necessario adottare un modello più sofisticato, capace di **catturare relazioni non lineari** e interazioni tra variabili. In questo contesto, un modello di **Gradient Boosting Regressor** risulta più adatto, grazie alla sua capacità di modellare pattern complessi e migliorare le prestazioni predittive rispetto alla semplice regressione lineare.


In [17]:
# Modello Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(random_state=42)
# Cross-validation (R²)
scores = cross_val_score(model, X, y, cv=10, scoring='r2')
print(f"R² Cross-Validation (Gradient Boosting): {scores.mean():.3f} ± {scores.std():.3f}")

# Addestramento su tutto il dataset
model.fit(X, y)

R² Cross-Validation (Gradient Boosting): 0.640 ± 0.110


GradientBoostingRegressor(random_state=42)

Un R² di 0.64 è già un grosso passo avanti rispetto allo 0.39 della regressione lineare, ma c'è sicuramente margine di miglioramento tramite hyperparameter tuning. 

In [18]:
from sklearn.model_selection import GridSearchCV 

# Parametri da testare
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [2, 3, 4],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Modello base
model = GradientBoostingRegressor(random_state=42)

# Grid search con cross-validation
grid_search = GridSearchCV(
    model,
    param_grid,
    cv=10,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

# Fit sul dataset completo
grid_search.fit(X, y)

# Migliori parametri trovati
print("Best parameters:", grid_search.best_params_)

# Miglior R² score cross-validated
print(f"Best CV R²: {grid_search.best_score_:.3f}")


Fitting 10 folds for each of 54 candidates, totalling 540 fits
Best parameters: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100, 'subsample': 1.0}
Best CV R²: 0.652


Non c'è questo gran miglioramento, proviamo un banale RandomForest tunato per gli hyperparameters:

In [19]:
from sklearn.ensemble import RandomForestRegressor

# Definizione dei parametri da testare
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'max_features': ['auto', 'sqrt'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Modello base
rf = RandomForestRegressor(random_state=42)

# GridSearchCV con 10-fold CV
grid_search_rf = GridSearchCV(
    rf,
    param_grid,
    cv=10,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

# Fit sul dataset completo
grid_search_rf.fit(X, y)

# Migliori parametri e score
print("Best parameters (RF):", grid_search_rf.best_params_)
print(f"Best CV R² (RF): {grid_search_rf.best_score_:.3f}")

Fitting 10 folds for each of 72 candidates, totalling 720 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
360 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
223 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.1

Best parameters (RF): {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best CV R² (RF): 0.676


Best parameters (RF): {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best CV R² (RF): 0.676 

### Motivazione della scelta del modello per la Web App

Dopo aver testato diversi modelli regressivi sul dataset, è emerso che il **Random Forest Regressor** è quello che garantisce le migliori prestazioni in termini di accuratezza e stabilità. 

Utilizzando solo le due variabili geografiche (`latitude` e `longitude`), il modello Random Forest ottimizzato ha raggiunto un **R² medio di 0.676** con una validazione incrociata a 10 fold, superando sia la regressione lineare semplice (R² ≈ 0.39) che il Gradient Boosting Regressor (R² ≈ 0.65).

Questi risultati evidenziano che il modello Random Forest:
- È in grado di **catturare relazioni non lineari** tra le coordinate geografiche e il prezzo al metro quadro.
- Offre una **migliore capacità predittiva**, pur utilizzando un numero limitato di feature.
- È più **robusto rispetto all’overfitting**, grazie all’aggregazione di più alberi decisionali.

Per questi motivi, si è scelto di utilizzare il **Random Forest Regressor** come modello di base all’interno della web app sviluppata con Streamlit.


In [20]:
# Salvataggio modello migliore
best_rf = grid_search_rf.best_estimator_

with open('model_lat_lon_rf.pkl', 'wb') as f:
    pickle.dump(best_rf, f)

## Web App con Stramlit 

In [21]:
import streamlit as st
import pickle
import numpy as np

# Carica il modello
with open('model_lat_lon_rf.pkl', 'rb') as f:
    model = pickle.load(f)

# Limiti geografici del dataset
LAT_MIN, LAT_MAX = 24.93, 25.08
LON_MIN, LON_MAX = 121.47, 121.56

st.title("Stima del Prezzo al Metro Quadro ")
st.write("Inserisci le coordinate geografiche per ottenere una stima del prezzo per unità di superficie nella regione di Sindian, Nuova Taipei (Taiwan).")

# Input utente
lat = st.number_input("Latitudine", min_value=LAT_MIN, max_value=LAT_MAX, format="%.6f")
lon = st.number_input("Longitudine", min_value=LON_MIN, max_value=LON_MAX, format="%.6f")

# Bottone per la previsione
if st.button("Stima il Prezzo"):
    input_data = np.array([[lat, lon]])
    predicted_price = model.predict(input_data)[0]
    st.success(f"Prezzo stimato: {predicted_price:.2f} NT$/m²")


2025-04-03 16:01:25.886 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 16:01:26.462 
  command:

    streamlit run /Users/nicolobachiorri/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-03 16:01:26.463 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 16:01:26.464 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 16:01:26.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 16:01:26.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 16:01:26.472 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-03 16:01:26.47